In [2]:
import quantecon as qe
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy import optimize

### Endogenous Grid Method

Problem:

#### Standard VFI Form:

$$V_{n+1}(k_i , z_j) = \max_{k'} \left\{\dfrac{(z_jk_i^\alpha + (1-\delta)k_i - k')^{1-\gamma}}{1-\gamma} + \beta \mathbb{E}( V_n(k',z') | z_j)\right\}$$
$$\text{s.t. } k' \geq k_{min}$$
$$ln(z') = \rho ln(z) + \eta'$$

#### EGM Form:

$$V_{n+1}(k , z_j) = \max_{k_i'} \left\{\dfrac{(z_jk^\alpha + (1-\delta)k - k_i')^{1-\gamma}}{1-\gamma} + \beta \mathbb{E}( V_n(k_i',z') | z_j)\right\}$$
$$\text{s.t. } ln(z') = \rho ln(z) + \eta'$$

So in the EGM form, we are creating grid for:
- capital grid for tomorrow's values
- Today's states

To understand why this method is better than standard VFI, we need to look at the FOC that we have to solve in standard VFI and compare it the FOC of EGM:

- Standard VFI: $(z_jk_i^\alpha + (1-\delta)k_i - k')^{-\gamma} = \beta \mathbb{E}(V_k(k',z')|z_j)$

- EGM: $z_k k^\alpha + (1-\delta)k = [\beta \mathbb{E}(V_k(k_i',z')|z_j)]^{\frac{-1}{\gamma}}+ k_i'$ 

Yes, even in EGM we need to solve for k (and it is a non-linear equation) but it is much easier to solve the first one for k' because we don't need to interpolate $\mathbb{E}(V_k(k',z')|z_j)$ for each off grid k' because all the k' s are on the grid (we chose the tomorrow's capital grid). Another beautiful thing is that instead of constantly evaluating expected value for tomorrow as in Standard VFI FOC, in EGM, it is enough to compute it once.

Furthermore, we can even avoid solving this non-linear equation by carefully changing the state variable from the beginning of period to end of period total **cash on hand**

Define: $Y = z_jk^\alpha + (1-\delta)k$

Then Belmann equation becomes:

$$\hat V(Y,z_j) = \max_{k'} \left\{ \dfrac{(Y-k')^{1-\gamma}}{1-\gamma} + \underbrace{\beta \mathbb{E}(\hat V(Y',z')|z_j)}_{\mathbb{V}(k_i',z_j)}\right\}$$
$$\text{s.t. } Y' = z'k'^\alpha + (1-\delta)k'$$
$$ k' \geq k_{min}$$
$$ln(z') = \rho ln(z) + \eta'$$

In [3]:
# Non stochastic economy:

# Compute the value function for the Non-stochastic economy first, and an obtain the policies. Them we can apply the algorithm for exercise a).

# Define Parameters:

n_k = 500  # Grids for capitals
n_A = 15  # Markov States.
δ = 0.9   # Depreciation.
α = 0.7   # Capital Share.
ρ = 0.98  # Memory of income
σ = 0.00  # Volatility of income.
β = 0.98  # Discont factor.
θ = 1.5   # Expanding grid coefficient.
error = 10e-6 # Error tolerance.
max_iter = 1000

# Construct the markov chain using rouwenhorst method:

markov = qe.markov.approximation.rouwenhorst(n= n_A, ybar = 1-ρ, sigma=σ, rho= ρ)

# Probability matrix and states:
# note that element (i,j) of P represents the transition probability
# of going to state "j" given that we are in state "i".

Π = markov.P   
A = markov.state_values  

# Maximum sustainable Capital:

K_max = 2
K_min = 0.1

# Use and expanding grid:

K = K_min + (K_max - K_min) * (np.linspace(0, 1, n_k)**θ)

# Utility function

def U(c):
    if (c>=0).all()>0.0001:
        return -1*((c)**(-1))
    else:
        return np.where(c <= 0.0001, np.nan, -1*(c)**(-1))

V_0 = np.ones((n_A, n_k))
V_1 = np.copy(V_0)
h_1 = np.ones((n_A, n_k), dtype=int)
iter = 0

while iter < max_iter:
    for r in range(n_A):
        for c in range(n_k):
            V_1[r,c] = np.nanmax(U( A[r] * K[c]**α + (1-δ) * K[c] - K) + β * Π[r,:] @ V_0)
            h_1[r,c] = np.nanargmax(U( A[r] * K[c]**α + (1-δ) * K[c] - K) + β * Π[r,:] @ V_0)
    m = 50
    for i in range(m):
        for r in range(n_A):
            for c in range(n_k):
                V_1[r,c] = U( A[r] * K[c]**α + (1-δ) * K[c] - K[h_1[r,c]]) + β * Π[r,:] @ V_1[:,h_1[r,c]]

    if np.amax(np.abs(V_1 - V_0))< error:
        print("Solution Found")
        print(f'Iterations', iter)
        print(f'Error', np.amax(np.abs(V_1 - V_0)))
        break
    else:
        V_0 = np.copy(V_1)
        iter += 1
        if iter >= max_iter:
            print("No solution")

VNSS = V_1.copy()
h_NSS = h_1.copy()

Solution Found
Iterations 20
Error 2.7959231374552473e-07


In [12]:

# Initial Values:

V_0 = VNSS.copy()
V_1 = np.copy(V_0)
h_1 = h_NSS.copy()
iter = 0

while iter < max_iter:
    for r in range(n_A):
        for c in range(n_k):
            V_1[r,c] = np.nanmax(U( A[r] * K[c]**α + (1-δ) * K[c] - K) + β * Π[r,:] @ V_0)
            h_1[r,c] = np.nanargmax(U( A[r] * K[c]**α + (1-δ) * K[c] - K) + β * Π[r,:] @ V_0)

    m = 10
    for i in range(m):
        for r in range(n_A):
            for c in range(n_k):
                V_1[r,c] = U( A[r] * K[c]**α + (1-δ) * K[c] - K[h_1[r,c]]) + β * Π[r,:] @ V_1[:,h_1[r,c]]        

    if np.amax(np.abs(V_1 - V_0))< error:
        print("Solution Found")
        print(f'Iterations', iter)
        print(f'Error', np.amax(np.abs(V_1 - V_0)))
        break

    #Howards steps: 
    
    else:
        V_0 = np.copy(V_1)
        iter += 1
        if iter >= max_iter:
            print("No solution")

In [9]:
for i,k in enumerate(Kp):
    for j,a in enumerate(A):
        expr = 

array([0.62394928, 0.67767081, 0.73139235, 0.78511388, 0.83883541,
       0.89255694, 0.94627847, 1.        , 1.05372153, 1.10744306,
       1.16116459, 1.21488612, 1.26860765, 1.32232919, 1.37605072])